In [55]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay


In [3]:
path = '/home/daisy/FDA_Dataset/inpatient_all_final_1.csv'
df1 = pd.read_csv(path).iloc[:,1:]
df1.drop(columns = ['Veteran flag','period mean','period std',
                    'Event date','Marital status', 'Marital status encoded',
                    'State','Ruca category'], inplace=True)


path = '/home/daisy/FDA_Dataset/inpatient_all_final_2.csv'
df2 = pd.read_csv(path).iloc[:,1:]
df2.drop(columns = ['Veteran flag','period mean','period std',
                    'Event date','Marital status', 'Marital status encoded',
                    'State','Ruca category'], inplace=True)

In [12]:
df1.shape

(84536, 81)

In [35]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84536 entries, 0 to 84535
Data columns (total 81 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Internalpatientid             84536 non-null  int64  
 1   num_stays                     84536 non-null  int64  
 2   stay_length                   84536 non-null  float64
 3   num_unique_units              84536 non-null  int64  
 4   num_transfers                 84536 non-null  int64  
 5   num_cvd_readmission           84536 non-null  int64  
 6   Readmission                   84536 non-null  int64  
 7   Died                          84536 non-null  int64  
 8   AO                            84536 non-null  int64  
 9   CVD                           84536 non-null  int64  
 10  unique_admitting_specialty    84536 non-null  int64  
 11  unique_discharging_specialty  84536 non-null  int64  
 12  DOMICILIARY                   84536 non-null  int64  
 13  M

### Train test split


In [26]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84536 entries, 0 to 84535
Data columns (total 81 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Internalpatientid             84536 non-null  int64  
 1   num_stays                     84536 non-null  int64  
 2   stay_length                   84536 non-null  float64
 3   num_unique_units              84536 non-null  int64  
 4   num_transfers                 84536 non-null  int64  
 5   num_cvd_readmission           84536 non-null  int64  
 6   Readmission                   84536 non-null  int64  
 7   Died                          84536 non-null  int64  
 8   AO                            84536 non-null  int64  
 9   CVD                           84536 non-null  int64  
 10  unique_admitting_specialty    84536 non-null  int64  
 11  unique_discharging_specialty  84536 non-null  int64  
 12  DOMICILIARY                   84536 non-null  int64  
 13  M

In [4]:
X_admission = df1.drop(columns = ['Readmission'])
Y_admission = df1[['Readmission']]

X_mortality = df1.drop(columns = ['Died'])
Y_mortality = df1[['Died']]

In [5]:
X_train_ad, X_test_ad, y_train_ad, y_test_ad = train_test_split(X_admission, Y_admission, test_size=0.20, random_state=42)
X_train_mor, X_test_mor, y_train_mor, y_test_mor = train_test_split(X_mortality, Y_mortality, test_size=0.20, random_state=42)


#### Filling missing values

In [8]:
df1.columns[df1.isna().any()].tolist()

['total_procedure',
 'num_surgery_pro',
 'num_immunization',
 'Num med per admission mean',
 'Num med per admission min',
 'Num med per admission max',
 'Total medications',
 'mean age at specailty',
 'specialty medical count',
 'specialty support count',
 'specialty count',
 'Age 20-40 hypotension',
 'Age 40-60 hypotension',
 'Age 60-80 hypotension',
 'Age 80-100 hypotension',
 'Age 100-120 hypotension',
 'Age 20-40 hypertension',
 'Age 40-60 hypertension',
 'Age 60-80 hypertension',
 'Age 80-100 hypertension',
 'Age 100-120 hypertension',
 'Age 20-40 healthy',
 'Age 40-60 healthy',
 'Age 60-80 healthy',
 'Age 80-100 healthy',
 'Age 100-120 healthy',
 'lab_count',
 'lab_freq',
 'lab_age_mean',
 'lab_age_std']

In [23]:
# Filling continuous features with mean values
numeric_cols = df1.columns[df1.isna().any()].tolist()

X_train_ad[numeric_cols] = X_train_ad[numeric_cols].fillna(X_train_ad[numeric_cols].mean())
X_train_mor[numeric_cols] =  X_train_mor[numeric_cols].fillna(X_train_mor[numeric_cols].mean())

X_test_ad[numeric_cols] = X_test_ad[numeric_cols].fillna(X_test_ad[numeric_cols].mean())
X_test_mor[numeric_cols] = X_test_mor[numeric_cols].fillna(X_test_mor[numeric_cols].mean())

In [22]:
X_train_ad.isna().any().sum()

0

#### Logistic Regression

In [24]:
lr =LogisticRegression()
# Training
lr.fit(X_train_ad, y_train_ad)

# Prediction
lr_prediction = lr.predict(X_test_ad)

/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [31]:
print(classification_report(y_test_ad, lr_prediction, target_names= ['Not Readmitted', 'Readmitted']))


                precision    recall  f1-score   support

Not Readmitted       0.87      0.85      0.86      3316
    Readmitted       0.96      0.97      0.97     13592

      accuracy                           0.95     16908
     macro avg       0.92      0.91      0.91     16908
  weighted avg       0.95      0.95      0.95     16908



In [41]:
lr = LogisticRegression()
# Training
lr.fit(X_train_mor, y_train_mor)

# Prediction
lr_prediction = lr.predict(X_test_mor)

/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [43]:
print(classification_report(y_test_mor, lr_prediction, target_names= ['Not Died', 'Died']))


              precision    recall  f1-score   support

    Not Died       0.83      0.98      0.90     13744
        Died       0.64      0.13      0.22      3164

    accuracy                           0.82     16908
   macro avg       0.73      0.56      0.56     16908
weighted avg       0.79      0.82      0.77     16908



In [51]:
y_train_ad['Readmission'].unique()

array([1, 0])

In [59]:
import statsmodels.api as sm
logit_model=sm.Logit(y_train_mor,X_train_mor)
result=logit_model.fit()
print(result.summary2()) 

Optimization terminated successfully.
         Current function value: 0.371561
         Iterations 9
                                     Results: Logit
Model:                      Logit                    Pseudo R-squared:         0.224     
Dependent Variable:         Died                     AIC:                      50389.8097
Date:                       2023-07-17 16:08         BIC:                      51000.9687
No. Observations:           67628                    Log-Likelihood:           -25128.   
Df Model:                   66                       LL-Null:                  -32361.   
Df Residuals:               67561                    LLR p-value:              0.0000    
Converged:                  1.0000                   Scale:                    1.0000    
No. Iterations:             9.0000                                                       
-----------------------------------------------------------------------------------------
                              Coef. 